# Verificar quais são os produtos que saem mais juntos em um mesmo atendimento

In [ ]:
import pandas as pd
import numpy as np
import sys, os

## 1. Criar os grupos

In [ ]:
# criar base vazia
df = pd.DataFrame(data = {'hash_atendimento':'', 'produto':''})

# criar uma base com os atendimentos que possuem mais de 2 produtos
hashes = df['hash_atendimento'].value_counts().reset_index()
hashes = hashes[hashes['count'] > 1].sort_values('hash_atendimento').reset_index(drop = True).reset_index(drop = False).drop('count', axis = 1)

# criar um loop para quebrar a base de 50k em 50k atendimentos
for index in range(21):

    min = index * 50000
    max = (index + 1) * 50000
    base_name = f'hashes_grupo_{str(index + 1)}'

    if index == 20:
        globals()[base_name] = hashes[hashes['index'] >= min]
    else:
        globals()[base_name] = hashes[(hashes['index'] >= min) & (hashes('index') < max)]

## 2. Rodar para todos os grupos

In [ ]:
for i in [*range(1, 22, 1)]:

    # filtrar apenas os atendimentos do grupo da iteração
    grupo = eval(f'hashes_grupo_{str(i)}')
    df_grupo = df.copy().merge(grupo[['hash_atendimento']], on = 'hash_atendimento', how = 'inner')

    # merge com a própria base para pegar as combinações
    df_grupo = df_grupo.merge(df_grupo, on = 'hash_atendimento', how = 'left')

    # remover linhas com x e y iguais
    df_grupo['manter'] = np.where(df_grupo['produto_x'] == df_grupo['produto_y'], False, True)
    df_grupo = df_grupo[df_grupo['manter']].drop('manter', axis = 1)

    # criar lista de pares e ordena-los
    df_grupo['lista'] = df_grupo.apply(lambda row: [row['produto_x'], row['produto_y']], axis = 1)
    df_grupo['lista'] = df_grupo['lista'].apply(lambda x: sorted(x))
    df_grupo['produto_x'] = df_grupo['lista'].apply(lambda x: x[0])
    df_grupo['produto_y'] = df_grupo['lista'].apply(lambda x: x[1])
    df_grupo = df_grupo.drop('lista', axis = 1).drop_duplicates()

    # contar quantas vezes as duplas aparecem
    df_grupo = df_grupo[['produto_x', 'produto_y']].value_counts().reset_index()

    nome_base = f'atendimentos_grupo_{str(i)}.parquet'

    df_grupo.to_parquet(os.path.join('caminho', nome_base))
    del(df_grupo)

## 3. Juntar todas as bases

In [ ]:
produtos_adq_juntos = pd.DataFrame()

for i in [*range(1, 22, 1)]:
    base = f'atendimentos_grupo_{str(i)}.parquet'
    df = pd.read_parquet(os.path.join('caminho', nome_base))

    produtos_adq_juntos = pd.concat([produtos_adq_juntos, df])

print(f'Tamanho da base concatenada: {produtos_adq_juntos.shape[0]}')
produtos_adq_juntos = produtos_adq_juntos.groupby(['produto_x', 'produto_y']).agg({'count':'sum'}).reset_index().sort_values('count', ascending = False)

print(f'Tamanho da base agregada: {produtos_adq_juntos.shape[0]}')

produtos_adq_juntos